In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.distributions import Multinomial
import biom
from biom import load_table


In [37]:
class MMVec(nn.Module):
    def __init__(self, num_microbes, num_metabolites, latent_dim):
        super().__init__()
        self.encoder = nn.Embedding(num_microbes, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, num_metabolites),
            # [batch, sample, metabolite]
            nn.Softmax(dim=2)
        )
        
    def forward(self, X, Y):
        z = self.encoder(X)
        y_pred = self.decoder(z)
        
        # total_count=0 and validate_args=False allows skipping total count when calling log_prob. Were having floating point issues with support.
        lp = Multinomial(total_count=0, validate_args=False, probs=y_pred).log_prob(Y).sum(0).sum()
        return lp

Getting data prepped:

In [3]:
# manually to start
microbes = load_table("./soil_microbes.biom")
metabolites = load_table("./soil_metabolites.biom")

# X = microbes.to_dataframe().T
# Y = metabolites.to_dataframe().T
# X = X.loc[Y.index]

# trainX = X.iloc[:-2]
# trainY = Y.iloc[:-2]
# testX = X.iloc[-2:]
# testY = Y.iloc[-2:]

# # index dictionaries for the inputs
# microbeIdxs = {microbe: i for i, microbe in enumerate(trainX)}
# metaboliteIdxs = {metabolite: i for i, metabolite in enumerate(trainY)}

# print(microbeIdxs, "\n------\n", metaboliteIdxs, "\n----\n", Y)
# exX = torch.tensor(trainX.values, requires_grad=True)
# exY = torch.tensor(trainY.values, requires_grad=True)


#### put the data in a dataloader(pytorch iterator)

In [40]:
class MicrobeMetaboliteData(Dataset):
    def __init__(self, microbes: biom.table, metabolites: biom.table, num_test_samples):
        # arrange
        self.microbes = microbes.to_dataframe().T   
        self.metabolites = metabolites.to_dataframe().T
        
        # only samples that have results
        self.microbes = self.microbes.loc[self.metabolites.index]
        
        # microbe index
#         self.microbe_idxs = {microbe: i for i, microbe in enumerate(self.microbes)}
        
        # make tensors
        self.microbes = torch.tensor(self.microbes.values, dtype=torch.int)
        # dtype must be integer to avoid floating point errors in multinomial
        self.metabolites = torch.tensor(self.metabolites.values, dtype=torch.int64)
        
        # inputs in dict so indexs available...not sure if we need this
        self.metabolite_idxs = {i: sample for i, sample in enumerate(self.metabolites)}
        
        self.microbe_count = self.microbes.shape[1]
        self.metabolite_count = self.metabolites.shape[1]
        
        self.metabolites_rel_freq = microbe_relative_frequency = (
            self.metabolites.T/self.metabolites.sum(1)).T
        
       
    def __len__(self):
        return len(self.microbes)
    
#     def __iter__(self):
#         # What do we want out?
#         # x random feature indexes, where x = the total number of counts in sample
#         # the expected outputs for this sample
        
#         # get the total number of observed features in the sample
# #         sample_microbes = self.microbes[idx]
# #         sample_microbe_obs = sample_microbes.sum()
        
#         # generate indexes to feed to the embedding
#         relative_frequency = (train_dataset.microbes.T/train_dataset.microbes.sum(1)).T
#         batch_multinomial = torch.multinomial(relative_frequency, 50).T
       
#         return batch_multinomial, self.metabolites


In [95]:
class MicrobeMetaboliteIterData(IterableDataset):
    def __init__(self, microbes: biom.table, metabolites: biom.table):
        # arrange
        self.microbes = microbes.to_dataframe().T   
        self.metabolites = metabolites.to_dataframe().T
        
        # only samples that have results
        self.microbes = self.microbes.loc[self.metabolites.index]
      
        # make tensors
        self.microbes = torch.tensor(self.microbes.values, dtype=torch.int)
        self.metabolites = torch.tensor(self.metabolites.values, dtype=torch.int64)
        
        # counts
        
        self.microbe_count = self.microbes.shape[1]
        self.metabolite_count = self.metabolites.shape[1]
        
        # relative frequencies
        self.microbe_relative_frequency = (self.microbes.T
                                      / self.microbes.sum(1)
                                     ).T
        
        self.metabolite_relative_frequency = (self.metabolites.T
                                     / self.metabolites.sum(1)
                                    ).T
        
        self.total_microbe_observations = self.microbes.sum()
        self.start = 0
        self.end = self.total_microbe_observations.item()
    def __len__(self):
        return self.microbe.shape[0]
    
    def __iter__(self):
        for i in range(self.start, self.end):
            yield torch.multinomial(self.microbe_relative_frequency, 500, replacement=True).T
        

#     def __getitem__(self, idx):
#         return self.microbe_relative_frequency

# def collater(data):
#     torch.multinomial(data, 500, replacement=True).T

In [97]:
train_dataset = MicrobeMetaboliteData(microbes, metabolites)
# train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)

# test_dataset = MicrobeMetaboliteData(testX, testY)
# test_dataloader = DataLoader(test_dataset, batch_size=8,
#                                  shuffle=True)
playset = MicrobeMetaboliteIterData(microbes, metabolites)

play_dataloader = DataLoader(playset, batch_size=500, shuffle=False)


# torch.multinomial(playset.microbe_relative_frequency, batch_size, replacement=True).shape
for i in play_dataloader:
    print(i)

tensor([[[419,  54,  69,  ...,   3, 202, 138],
         [  0,  81, 114,  ..., 435, 413, 347],
         [415, 195, 291,  ...,  15, 414, 335],
         ...,
         [ 86, 347, 299,  ...,  80,  64,   1],
         [455, 456,   1,  ..., 330, 291, 225],
         [  0,   0, 176,  ...,  92,  92, 224]],

        [[  0, 202,   1,  ...,  15, 302, 458],
         [ 41,   1,  92,  ..., 103, 126, 147],
         [ 81, 447,   1,  ..., 103, 147, 455],
         ...,
         [224, 438, 435,  ..., 103,   3, 224],
         [454,  38, 125,  ..., 103, 434,  37],
         [  0,   1,  61,  ..., 285, 335, 436]],

        [[125,   1, 138,  ..., 437, 202, 324],
         [ 96, 431,   1,  ..., 437, 313, 224],
         [180, 136, 200,  ...,  85,  96, 128],
         ...,
         [  0,   0,   0,  ..., 147,  52, 440],
         [  0,  59, 314,  ..., 285, 146, 224],
         [361,   0, 170,  ..., 424,  59, 440]],

        ...,

        [[191,   0,  18,  ..., 147, 213,  92],
         [369,   0,   1,  ..., 103, 335, 224]

tensor([[[  0,  59, 257,  ...,   3,  39, 421],
         [ 99,   0, 256,  ..., 202,  65, 441],
         [105, 449,  26,  ...,  97, 105, 224],
         ...,
         [ 17, 103, 335,  ..., 424,  81, 224],
         [279,   1, 114,  ..., 103, 221, 440],
         [358, 142, 235,  ..., 451,   2, 224]],

        [[  5,  70, 263,  ..., 250, 465, 246],
         [ 98,   0, 136,  ..., 103,  13, 258],
         [439, 191,   0,  ..., 103,  64, 181],
         ...,
         [135, 418,   0,  ..., 103,  58,   3],
         [ 26,  70,   4,  ..., 236, 132, 224],
         [  7, 438, 459,  ..., 424, 173, 208]],

        [[439, 129,   0,  ..., 193, 335, 402],
         [  0,   1,  49,  ...,   0, 335, 410],
         [337,   1,  34,  ..., 438, 441, 146],
         ...,
         [ 48, 336, 291,  ..., 103, 433, 202],
         [203,   0, 125,  ...,  39, 126, 197],
         [ 86,   1, 200,  ..., 202, 165, 421]],

        ...,

        [[131,   1,  84,  ..., 245, 335, 336],
         [  0,  72,   0,  ..., 103,  79, 224]

tensor([[[358,  87, 258,  ..., 197, 335, 224],
         [443, 393,   1,  ...,  97, 224, 369],
         [  0,  68,  91,  ..., 424,  45, 440],
         ...,
         [169,  31, 100,  ...,   0, 461, 258],
         [418, 457,   0,  ...,  97, 335, 378],
         [101, 431, 335,  ..., 103,   1, 419]],

        [[  0, 191,   1,  ..., 103, 451,   9],
         [439,   0,   1,  ..., 424, 435, 335],
         [418,   0, 153,  ..., 224, 423, 440],
         ...,
         [ 92,   0, 224,  ..., 103,  82, 112],
         [225, 348,   0,  ..., 189, 414, 258],
         [439,  57, 314,  ..., 103,  39, 105]],

        [[412,   1,   1,  ..., 302, 295, 335],
         [374,  59, 444,  ..., 437, 202, 224],
         [439,  81, 237,  ..., 103, 185,  70],
         ...,
         [455, 439, 186,  ...,  25, 114, 224],
         [  5,  32, 180,  ..., 430, 426,   5],
         [  0, 419,   0,  ..., 250, 111, 224]],

        ...,

        [[180, 216, 449,  ..., 193, 243, 171],
         [ 17, 380,   0,  ..., 103, 414, 426]

tensor([[[428, 132,   3,  ..., 103, 417, 420],
         [105, 423, 258,  ..., 261, 441, 146],
         [147,  70, 464,  ..., 430, 418, 441],
         ...,
         [275, 452,   9,  ..., 114, 173, 421],
         [447,  70,   0,  ...,  39,  92, 426],
         [433, 428, 459,  ..., 455,  79,  86]],

        [[414,  84,   1,  ..., 103,  92, 451],
         [ 25,  92, 413,  ..., 103, 451, 140],
         [425,  84, 413,  ...,   0,  70, 440],
         ...,
         [430, 280, 413,  ...,  45, 164, 440],
         [158, 313,   8,  ...,  85, 192, 436],
         [284,   1, 459,  ..., 302, 413, 440]],

        [[422,   0, 335,  ..., 236, 258, 224],
         [366, 280, 152,  ...,  15,  72, 224],
         [324, 402, 347,  ..., 102, 335, 440],
         ...,
         [279, 184,   1,  ..., 420,  15, 224],
         [107,   1,   1,  ...,  58, 224, 418],
         [ 48,   3, 464,  ..., 103, 402, 242]],

        ...,

        [[174, 439, 335,  ..., 431,   3, 437],
         [  0,   1, 435,  ..., 109, 335, 335]

tensor([[[269,   0,   1,  ..., 103, 258,  37],
         [  0,  59,  48,  ..., 103, 330, 437],
         [  0,   0, 429,  ..., 103, 335, 348],
         ...,
         [433,  81, 334,  ..., 103, 451, 440],
         [207, 391, 294,  ..., 192, 204,  37],
         [ 59,  77,   4,  ..., 338, 464, 436]],

        [[191,  20, 429,  ..., 103, 249, 436],
         [139, 125,  48,  ...,  15,  40, 440],
         [  0, 455,   0,  ...,  64,  26, 258],
         ...,
         [258, 156,   0,  ...,  59,   1, 440],
         [  0,  72,   0,  ...,   0, 213, 224],
         [186, 288, 335,  ..., 302,  59, 224]],

        [[431,   1,   0,  ...,  97, 417, 440],
         [  0,  70, 131,  ..., 103, 441,  35],
         [458,  70,   3,  ..., 425, 441, 455],
         ...,
         [169,  58,   0,  ...,  81,  40, 365],
         [  9, 358,   1,  ..., 424,  23, 440],
         [ 13,   1, 279,  ..., 103,   0, 173]],

        ...,

        [[251, 180,  48,  ..., 448, 158, 440],
         [447,   1,   0,  ..., 265, 354, 202]

...

tensor([[[ 95,  35,   1,  ...,  59, 354, 224],
         [  0,  80,   0,  ..., 103, 413, 224],
         [336,   1,   0,  ..., 102, 213, 440],
         ...,
         [184,  70,   1,  ...,  64,  38,   9],
         [258,   1, 335,  ..., 147, 124, 335],
         [414, 252,   3,  ..., 426,   7, 441]],

        [[169, 223,   0,  ..., 437, 447, 147],
         [ 60,   3, 410,  ..., 437, 147, 437],
         [ 92, 262, 179,  ..., 171, 461, 224],
         ...,
         [380, 423,  65,  ..., 424, 335, 224],
         [ 81,   1,   1,  ..., 376, 137,  62],
         [246,  58, 366,  ..., 103, 224,   3]],

        [[309, 199,   0,  ..., 459,   9, 436],
         [350,   1,   0,  ...,   0, 161, 224],
         [428, 249, 202,  ..., 136, 105, 224],
         ...,
         [439,  31, 335,  ..., 321, 413, 224],
         [439,  34, 144,  ..., 103, 224,  37],
         [313,   1, 428,  ..., 103, 185, 181]],

        ...,

        [[ 70,  34, 151,  ...,   0, 278, 147],
         [ 13,  26,  48,  ..., 430, 215, 436]

tensor([[[439, 156,   9,  ...,  25, 421, 139],
         [131, 142,  48,  ..., 245, 413, 147],
         [ 84, 158, 224,  ...,  53,  89, 335],
         ...,
         [428, 423,   1,  ..., 424, 335, 246],
         [425,   1,   4,  ...,   2, 279,  29],
         [  0,  50,  48,  ..., 103, 391, 324]],

        [[184, 455,   1,  ..., 202,   4, 440],
         [246,  26,   0,  ..., 324,  30, 436],
         [ 13,  15, 206,  ..., 165, 109,  15],
         ...,
         [151,  54, 118,  ..., 197, 413, 224],
         [444,   1, 335,  ...,   3, 335,  26],
         [202,   1, 455,  ...,  97, 208, 425]],

        [[ 17, 336, 433,  ..., 103, 128, 410],
         [ 41,  10,   0,  ..., 103,   5,  65],
         [428, 431,   0,  ..., 103, 147,  37],
         ...,
         [336,  81,   0,  ..., 103, 259, 224],
         [104,   1, 271,  ...,  26, 136, 436],
         [286, 288,  24,  ..., 424, 411, 224]],

        ...,

        [[443, 455,   1,  ...,  26, 154,  51],
         [191, 411, 299,  ..., 147,  39, 191]

tensor([[[439, 142,   1,  ..., 107, 158, 202],
         [  0,  84, 183,  ..., 445, 336, 258],
         [  0,   1,   7,  ..., 317,  26,  92],
         ...,
         [460,   1,  48,  ...,  70,  79, 436],
         [228,   3, 325,  ...,  13,  30, 440],
         [391, 350,   0,  ..., 324, 444,   1]],

        [[ 69,   1, 410,  ..., 147, 402,  72],
         [444, 439,  78,  ...,   3, 315, 258],
         [174,   5, 430,  ..., 451, 160, 369],
         ...,
         [231, 358,   1,  ..., 227, 459, 224],
         [428,  96,  93,  ...,  68, 114,  29],
         [426, 439, 422,  ..., 186, 186, 224]],

        [[444, 143,  48,  ..., 440, 224, 224],
         [444,  38, 413,  ...,   0, 413,  15],
         [  0,  87, 445,  ...,  51, 158, 224],
         ...,
         [  0,  70,   0,  ..., 103,   1,  86],
         [180,  84,  56,  ..., 185, 456, 365],
         [  0, 431, 396,  ..., 420, 335, 426]],

        ...,

        [[402, 252,  12,  ..., 103, 335, 369],
         [  0,   0, 297,  ..., 103, 224, 147]

tensor([[[104, 456, 258,  ..., 261, 369, 335],
         [  0, 455, 225,  ..., 424, 441,   3],
         [  0,   0, 127,  ...,  32, 264, 224],
         ...,
         [169, 358,  70,  ..., 465,  39, 202],
         [454,   1,  54,  ..., 147, 213, 465],
         [357,  99,   4,  ..., 103, 352, 202]],

        [[ 48,   0,   1,  ...,  97,  23,  90],
         [153, 225, 328,  ...,   0, 189,  15],
         [ 81,   3, 369,  ..., 114,  90, 425],
         ...,
         [ 59,   1,   1,  ..., 147, 109,  31],
         [ 81, 362,  48,  ..., 103, 224,  41],
         [158,   1,   0,  ..., 424, 423, 449]],

        [[  0,  70,  48,  ..., 161, 252,  15],
         [444,   1,   1,  ..., 414, 434, 436],
         [ 13, 281,   8,  ...,   7,  80,  52],
         ...,
         [345, 412,  89,  ...,   0, 128, 265],
         [424, 460,   4,  ..., 103, 173, 224],
         [  5,   1, 462,  ...,  97, 391, 224]],

        ...,

        [[455, 248, 179,  ...,   0,  30, 436],
         [434,   1,  69,  ..., 424, 414, 224]

tensor([[[  0,  92, 295,  ...,  59,  40, 224],
         [425,  73,  48,  ...,  20,  79, 436],
         [  0, 455, 442,  ..., 103, 412, 335],
         ...,
         [  0,  84,   8,  ..., 459,  79, 111],
         [  0, 336, 115,  ...,   2,  51, 438],
         [169,  70,  33,  ..., 103, 349, 138]],

        [[  0,  12, 429,  ..., 437, 226,  58],
         [224, 435, 429,  ..., 103, 354, 129],
         [  0, 455,   4,  ..., 115,   1, 436],
         ...,
         [  0,  73, 136,  ..., 187, 249, 436],
         [  0, 347, 225,  ..., 185, 125, 208],
         [181, 419,   0,  ..., 266, 351, 224]],

        [[ 48, 455,  48,  ..., 358, 413, 224],
         [280,  38, 145,  ..., 424, 354, 401],
         [  0, 168,   1,  ..., 411,   0, 380],
         ...,
         [  0,  92,   1,  ..., 261, 258,  37],
         [  0,  81, 396,  ..., 424, 204, 420],
         [121, 342,  48,  ..., 420,   7, 436]],

        ...,

        [[114,  26, 455,  ..., 451, 411, 147],
         [ 37,   1, 411,  ..., 424, 125, 141]

## Training...

In [17]:
learning_rate = 1e-3
batch_size = 500
epochs = 25

In [38]:
mmvec_model = MMVec(train_dataset.microbe_count, train_dataset.metabolite_count, 10)

optimizer = torch.optim.Adam(mmvec_model.parameters(), lr=learning_rate, maximize=True)

In [39]:
def train_loop(dataset, model, optimizer, batch_size):
    n_batches = torch.div(dataset.microbes.sum(),
                          batch_size,
                          rounding_mode = 'floor') + 1
    
    microbe_relative_frequency = (dataset.microbes.T/dataset.microbes.sum(1)).T
    
    for batch in range(n_batches * epochs):
    
        lp = model(torch.multinomial(microbe_relative_frequency, batch_size, replacement=True).T,
              dataset.metabolites_rel_freq)
    
        optimizer.zero_grad()
        lp.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            print(f"loss: {lp.item()}\nBatch #: {batch}")
        
        
    
train_loop(train_dataset, mmvec_model, optimizer, 500)

loss: -40077.8125
Batch #: 0
loss: -36404.13671875
Batch #: 100
loss: -31702.857421875
Batch #: 200
loss: -27709.734375
Batch #: 300
loss: -25352.328125
Batch #: 400
loss: -24233.96875
Batch #: 500
loss: -23553.99609375
Batch #: 600
loss: -23262.22265625
Batch #: 700
loss: -23128.380859375
Batch #: 800
loss: -23025.640625
Batch #: 900
loss: -22967.24609375
Batch #: 1000
loss: -22924.705078125
Batch #: 1100
loss: -22890.03515625
Batch #: 1200
loss: -22856.15234375
Batch #: 1300
loss: -22833.298828125
Batch #: 1400
loss: -22811.720703125
Batch #: 1500
loss: -22770.0
Batch #: 1600
loss: -22737.439453125
Batch #: 1700
loss: -22731.591796875
Batch #: 1800
loss: -22710.056640625
Batch #: 1900
loss: -22683.625
Batch #: 2000
loss: -22667.99609375
Batch #: 2100
loss: -22668.8203125
Batch #: 2200
loss: -22634.103515625
Batch #: 2300
loss: -22620.904296875
Batch #: 2400
loss: -22590.16015625
Batch #: 2500
loss: -22598.8515625
Batch #: 2600
loss: -22579.01171875
Batch #: 2700
loss: -22565.62890625

In [9]:
out1 = mmvec_model(torch.tensor([[0, 0, 0],
                                 [1, 1, 1]]), 
           train_dataset.metabolites[0:3, :])

In [10]:
out1

tensor(-2.6305e+10, grad_fn=<MeanBackward0>)

In [11]:
out1[0, 0] + out1[1, 0]

IndexError: too many indices for tensor of dimension 0

In [ ]:
out1[0, 1] + out1[1, 1]

In [ ]:
out1.mean(0)

In [ ]:
train_dataset.microbes.sum(1)

In [ ]:
relative_frequency = (train_dataset.microbes.T/train_dataset.microbes.sum(1)).T

In [ ]:
torch.multinomial(relative_frequency, 50).T

In [ ]:
batched_out = mmvec_model(torch.multinomial(relative_frequency, 50).T, train_dataset.metabolites)

In [ ]:
batched_out.mean(0).mean()